In [ ]:
# Imports
import investpy
import pandas as pd
import numpy as np
import yfinance as yf
import geopandas as gpd
from keplergl import KeplerGl
from pathlib import Path

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

# Import Wheat Data

In [ ]:
wheat_df = investpy.get_commodity_historical_data(commodity = 'US Wheat',country = 'united states',
                                                  from_date = '01/01/2012',to_date = '01/01/2022')
wheat_df.head(5)

# Editing the Data to specify the Close price and giving the colum name a more specific name.

In [ ]:
wheat_df = wheat_df.drop(['Open','High','Low','Volume','Currency'], axis=1)
wheat_df = wheat_df.rename(columns = {'Close': 'Wheat'})
wheat_df.head(5)

# Import Crude Data

In [ ]:
crude_df = investpy.get_commodity_historical_data(commodity = 'Crude Oil WTI', country = 'united states',
                                                  from_date = '01/01/2012',to_date = '01/01/2022')
crude_df.head(5)

# Edit and clean data

In [ ]:
crude_df = crude_df.drop(['Open','High','Low','Volume','Currency'],axis=1)
crude_df = crude_df.rename(columns = {'Close': 'Crude'})
crude_df.head(5)

# Import Carbon futures data

In [ ]:
carbon_df = pd.read_csv("../Data/Carbon Emissions Futures Historical Data.csv")
carbon_df.head(5)

# Edit Carbon data

In [ ]:
carbon_df.reset_index()
carbon_df.rename(columns={'Price':'Carbon Futures'})
carbon_df['Date'] = pd.to_datetime(carbon_df.Date)
carbon_df['Date'] = carbon_df['Date'].dt.strftime('%Y-%m-%d')

carbon_df.head(5)


In [ ]:
carbon_df = carbon_df.set_index("Date")
carbon_df.head(5)

In [ ]:
crude_df = crude_df.rename(columns = {'Price': 'Carbon'})
crude_df.head(5)

In [ ]:
crude_df.index = pd.to_datetime(crude_df.index, format = '%Y-%m-%d').strftime('%Y-%m-%d')
carbon_df.index = pd.to_datetime(carbon_df.index, format = '%Y-%m-%d').strftime('%Y-%m-%d')
wheat_df.index = pd.to_datetime(wheat_df.index, format = '%Y-%m-%d').strftime('%Y-%m-%d')

In [ ]:
crudecarbon = pd.merge(crude_df,carbon_df,on="Date", how='inner')
crudecarbon = crudecarbon.drop(['Open','High','Low','Vol.','Change %'],axis=1)
crudecarbon.rename(columns={'Price':'Carbon'}, inplace=True)
crudecarbon

In [ ]:
wheatcrudecarbon = pd.merge(crudecarbon,wheat_df, on='Date',how='inner')
wheatcrudecarbon

In [ ]:
wheatcrudecarbon.rename(columns={'Price':'Carbon'}, inplace=True)

In [ ]:
returns = wheatcrudecarbon.dropna()
returns.head(3)

In [ ]:
# Show correlation matrix
sns.heatmap(returns.corr(),cmap='viridis',annot=True,vmin=-1, vmax=1);

# Import Wheat Data

In [ ]:
# Get historical wheat price data from investpy
wheat_df = pd.read_csv("../Data/wheat_df-Update.csv",parse_dates=True,infer_datetime_format=True,index_col='Date')
                    
                      

# Import Copper Data

In [ ]:
# Get historical copper price data from investpy
copper_df =pd.read_csv("../Data/copper_df-Update.csv",parse_dates=True,infer_datetime_format=True,index_col='Date')

# Import Dollar_index Data

In [ ]:
# Get historical Dollar Index data from investpy
dollar_df = pd.read_csv("../Data/dollar_df-Update.csv",parse_dates=True,infer_datetime_format=True,index_col='Date')

# Import drought dataset

In [ ]:
#drought dataset
drought_df = pd.read_csv("../Data/drought_df-Update.csv",parse_dates=True,infer_datetime_format=True,index_col='Date')
drought_df.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1,inplace=True)

# Import Urea dataset

In [ ]:
# Urea dataset
Urea_df = pd.read_csv("../Data/Urea_df-Update.csv",parse_dates=True,infer_datetime_format=True,index_col='Date')

In [ ]:
# Combine data frames
combined_df = pd.concat([dollar_df,copper_df,Urea_df,drought_df,wheat_df],axis=1,join='inner')

In [ ]:
# Create correlation matrix
correlations_df = combined_df.corr()
correlations_df

In [ ]:
# Show correlation matrix
sns.heatmap(correlations_df,cmap='viridis',annot=True,vmin=-1,vmax=1);

# Import Wheat Data

In [ ]:
# Get historical wheat price data from investpy
wheat_df = pd.read_csv("../Data/wheat_df-Update.csv",parse_dates=True,infer_datetime_format=True,
                       index_col='Date')

# Import Crude Data

In [ ]:
# Get historical crude oil price data from investpy
crude_df = pd.read_csv("../Data/crude_df-Update.csv",parse_dates=True,infer_datetime_format=True,
                       index_col='Date')
   

# Import Carbon futures data

In [ ]:
# Get historical Carbon futures price data from investpy
Carbon_df = pd.read_csv("../Data/Carbon_df-Update.csv",parse_dates=True,infer_datetime_format=True,
                       index_col='Date')

# Combine data frames

In [ ]:
combined_df_ = pd.concat([crude_df,Carbon_df,wheat_df],axis=1,join='inner')

# Create correlation matrix

In [ ]:
correlations_df = combined_df_.corr()
correlations_df

In [ ]:
# Show correlation matrix
sns.heatmap(correlations_df,cmap='viridis',annot=True,vmin=-1,vmax=1);

In [ ]:
US = 'united states'
START = '01/01/1992'
END = '01/01/2022'

In [ ]:
# Get historical wheat price data from investpy
wheat_df = investpy.get_commodity_historical_data(commodity = 'US Wheat',
                                                  country = US,
                                                  from_date = START,
                                                  to_date = END)
wheat_df.drop(['Open','High','Low','Volume','Currency'], axis=1, inplace=True)
wheat_df.rename(columns = {'Close': 'Wheat'}, inplace=True)

In [ ]:
# Get historical Dollar Index data from investpy
dx_df = investpy.get_index_historical_data(index = 'US Dollar Index',
                                           country = US,
                                           from_date = START,
                                           to_date = END)
dx_df.drop(['Open','High','Low','Volume','Currency'], axis=1, inplace=True)
dx_df.rename(columns = {'Close': 'Dollar'}, inplace=True)

In [ ]:
# Get historical copper price data from investpy
copper_df = investpy.get_commodity_historical_data(commodity = 'Copper',
                                                   country = US,
                                                   from_date = START,
                                                   to_date = END)
copper_df.drop(['Open','High','Low','Volume','Currency'], axis=1, inplace=True)
copper_df.rename(columns = {'Close': 'Copper'}, inplace=True)

In [ ]:
# Create function to convert string date to datetime object
def convert_date_urea(date):
    return pd.to_datetime(date[-4:] + date[:4].strip(), format='%Y%b')

In [ ]:
# Import urea (nitrogen fertilizer) price data
urea_df = pd.read_html("https://www.indexmundi.com/commodities/?commodity=urea&months=360")[1].drop(columns='Change', axis=0)

In [ ]:
# Apply date conversion and reset index
urea_df.Month = urea_df.Month.apply(convert_date_urea)
urea_df.set_index('Month', inplace=True)

In [ ]:
# Rename column
urea_df.rename(columns = {'Price': 'Urea'}, inplace=True)

In [ ]:
# Create function to convert string date to datetime object
def convert_date_drought(date):
    date = str(date)
    return pd.to_datetime(date[:5] + date[5:], format='%Y%m')

In [ ]:
# Import Palmer Drought Severity Index data for the Great Plains region
drought_df = pd.read_csv('../Data/pdsi1992-2022.csv')

In [ ]:
# Apply date conversion and reset index
drought_df.Date = drought_df.Date.apply(convert_date_drought)
drought_df.set_index('Date', inplace=True)

In [ ]:
# Drop and rename columns
drought_df.drop(columns='Anomaly', axis=0, inplace=True)
drought_df.rename(columns = {'Value': 'Drought'}, inplace=True)


In [ ]:
# Combine data frames
combined_df = pd.concat([dx_df,copper_df,urea_df,drought_df,wheat_df], axis=1, join='outer')

In [ ]:
# Seperate out month and year from index
combined_df['Month'] = combined_df.index.month
combined_df['Year'] = combined_df.index.year


In [ ]:
# Fill nulls in Urea and Drought with monthly value and
# compute monthly average for Wheat, Dollar, and Copper
combined_df = combined_df.groupby(['Year','Month']).mean().dropna()
combined_df

In [ ]:
# Create correlation matrix
correlations_df = combined_df.corr()
correlations_df

In [ ]:
# Create mask for correlation heatmap
mask = np.triu(np.ones_like(correlations_df))
np.fill_diagonal(mask, 0)

In [ ]:
# Show correlation heatmap
fig, ax = plt.subplots(figsize=(5, 4),dpi=200)
sns.heatmap(correlations_df,cmap='RdBu',annot=True, vmin=-1, vmax=1, ax=ax, mask=mask)
ax.set_title('Correlation Between Wheat Prices and Selected Factors\n')
plt.show()

In [ ]:
# Kraft stock data
# Get historical wheat price data from investpy
khc_df = investpy.get_stock_historical_data(stock = 'KHC',
                                                  country = US,
                                                  from_date = START,
                                                  to_date = END)
khc_df.drop(['Open','High','Low','Volume','Currency'], axis=1, inplace=True)
khc_df.rename(columns = {'Close': 'KHC'}, inplace=True)

In [ ]:
# Show correlation heatmap
fig, ax = plt.subplots(figsize=(5, 4),dpi=200)
sns.lineplot(data=khc_df,ax=ax)
ax.set_title('Kraft Heinz (KHC) Stock Price 2009-2022\n')
plt.show()

In [59]:
countries = gpd.read_file("../Data/countries.geojson")
countries["Country"] = countries["ADMIN"]

NameError: name 'gpd' is not defined

In [ ]:
# Using the read_csv function and Path module, create a DataFrame 
wheat_exports = pd.read_csv(
    Path('./Wheat_Exports.csv'))
wheat_exports.head()

,Country,USD_value_wheat_billion,pct_global_wheat_exports
0,Russia,7.30,13.10%
1,United States,7.29,13.10%
2,Australia,7.20,13%
3,Canada,6.60,11.90%
4,Ukraine,4.70,8.50%


In [ ]:
# Using the read_csv function and Path module, create a DataFrame 
wheat_imports = pd.read_csv(
    Path('./Wheat_Imports.csv'))
wheat_imports.head()

,Rank,Country,Imports (1000 MT)
0,1,Indonesia,"11,200"
1,2,Egypt,"11,000"
2,3,Turkey,"10,000"
3,4,China,"9,500"
4,5,Algeria,"7,900"


In [ ]:
#wheat imports join/merge
countries_w_wheat_imports = pd.merge(countries, wheat_imports, on='Country')
countries_w_wheat_imports.head()

,ADMIN,ISO_A3,geometry,Country,Rank,Imports (1000 MT)
0,Afghanistan,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903...",Afghanistan,20,"3,400"
1,Angola,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -...",Angola,50,"1,000"
2,Albania,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993...",Albania,89,300
3,United Arab Emirates,ARE,"MULTIPOLYGON (((53.86305 24.23469, 53.88860 24...",United Arab Emirates,32,"1,800"
4,Argentina,ARG,"MULTIPOLYGON (((-68.65412 -54.88624, -68.65414...",Argentina,125,5


In [ ]:
#wheat exports join/merge
countries_w_wheat_exports = pd.merge(countries, wheat_exports, on='Country')
countries_w_wheat_exports.head()

,ADMIN,ISO_A3,geometry,Country,USD_value_wheat_billion,pct_global_wheat_exports
0,Argentina,ARG,"MULTIPOLYGON (((-68.65412 -54.88624, -68.65414...",Argentina,3.0,5.30%
1,Australia,AUS,"MULTIPOLYGON (((158.86573 -54.74993, 158.83823...",Australia,7.2,13%
2,Bulgaria,BGR,"POLYGON ((22.91956 43.83422, 23.05255 43.84282...",Bulgaria,1.4,2.50%
3,Canada,CAN,"MULTIPOLYGON (((-65.61059 43.42817, -65.62881 ...",Canada,6.6,11.90%
4,Germany,DEU,"MULTIPOLYGON (((6.74220 53.57836, 6.74952 53.5...",Germany,2.0,3.60%


## Wheat Imports Visualization

In [ ]:
wheat_imports_map = KeplerGl()
wheat_imports_map.add_data(data=countries_w_wheat_imports, name="wheat_imports")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [ ]:
wheat_imports_map.show()

In [ ]:
wheat_imports_map.config

{}

## Wheat Exports Visualization

In [ ]:
wheat_exports_map = KeplerGl()
wheat_exports_map.add_data(data=countries_w_wheat_exports, name="wheat_exports")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [ ]:
wheat_exports_map.show()